In [ ]:
import os
import pandas as pd

# 데이터 파일이 있는 디렉토리 경로
directory_path = '/content/drive/MyDrive/외국인관광객지출/소비추이'

# 변환된 데이터를 저장할 리스트
all_data = []

# 디렉토리의 모든 파일 처리
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):  # CSV 파일만 처리
        file_path = os.path.join(directory_path, filename)

        # 파일 읽기
        data = pd.read_csv(file_path, encoding='euc-kr')

        # 피벗 변환
        pivoted_data = data.pivot_table(
            index='기준년월',
            columns=['중분류'],
            values='소비액(천원)',
            aggfunc='sum'
        ).reset_index()

        # 컬럼 이름 정리
        pivoted_data.columns.name = None

        # 변환된 데이터를 리스트에 추가
        all_data.append(pivoted_data)

# 모든 데이터를 하나로 합치기
final_data = pd.concat(all_data, ignore_index=True)

# 중복된 데이터 제거
final_data = final_data.drop_duplicates(subset='기준년월')

# 값이 없는 부분을 0으로 채우기
final_data = final_data.fillna(0)

# '관광총소비' 컬럼을 마지막으로 이동
if '관광총소비' in final_data.columns:
    columns = [col for col in final_data.columns if col != '관광총소비'] + ['관광총소비']
    final_data = final_data[columns]

# 결과를 파일로 저장
output_path = '/content/서울외국인관광객소비.csv'  # 저장 경로 설정
final_data.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"모든 파일이 처리되었고, 결과가 {output_path}에 저장되었습니다.")


모든 파일이 처리되었고, 결과가 /content/서울외국인관광객소비.csv에 저장되었습니다.


In [ ]:
import pandas as pd

# 파일 경로 설정
domestic_file_path = '/content/서울관광객소비.csv'  # 내국인 파일 경로
foreign_file_path = '/content/서울외국인관광객소비.csv'  # 외국인 파일 경로

# 1. 내국인 데이터 읽기
domestic_data = pd.read_csv(domestic_file_path, encoding='utf-8')
domestic_data['구분'] = '내국인'  # 내국인 구분 컬럼 추가

# 2. 외국인 데이터 읽기
foreign_data = pd.read_csv(foreign_file_path, encoding='utf-8')
foreign_data['구분'] = '외국인'  # 외국인 구분 컬럼 추가

# 3. 두 데이터를 병합 (행 추가)
combined_data = pd.concat([domestic_data, foreign_data], ignore_index=True)

# 4. 데이터 정렬
# '구분'을 기준으로 먼저 정렬하고, 같은 구분 내에서는 '기준년월'을 기준으로 정렬
combined_data = combined_data.sort_values(by=['구분', '기준년월']).reset_index(drop=True)

# 5. 컬럼 순서 변경
# 기준년월 뒤에 '구분'을 배치하고, 관광총소비를 마지막으로 이동
columns = ['기준년월'] + ['구분'] + [col for col in combined_data.columns if col not in ['기준년월', '구분', '관광총소비']] + ['관광총소비']
combined_data = combined_data[columns]

# 값이 없는 부분을 0으로 채우기
combined_data = combined_data.fillna(0)

# 6. 결과 저장
output_path = '/content/서울관광객내외병합_정렬.csv'  # 저장 경로 설정
combined_data.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"내국인과 외국인 데이터를 병합하고 날짜 기준으로 정렬한 결과가 {output_path}에 저장되었습니다.")


내국인과 외국인 데이터를 병합하고 날짜 기준으로 정렬한 결과가 /content/서울관광객내외병합_정렬.csv에 저장되었습니다.


In [ ]:
import pandas as pd

# 파일 경로 설정
temperature_file_path = '/content/서울기온.csv'
output_file_path = '/content/정리된_서울기온.csv'  # 저장할 파일 경로

# 1~7번째 줄 제거하고 데이터 읽기
temperature_data = pd.read_csv(temperature_file_path, encoding='euc-kr', skiprows=7)

# 날짜에서 '-' 제거
temperature_data['날짜'] = temperature_data['날짜'].str.strip().str.replace('-', '')

# 필요한 열만 선택 및 열 이름 수정
temperature_data = temperature_data[['날짜', '평균기온(℃)', '최저기온(℃)', '최고기온(℃)']]
temperature_data.rename(columns={'날짜': '기준년월'}, inplace=True)

# 변환된 데이터 저장
temperature_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"정리된 데이터가 {output_file_path}에 저장되었습니다.")


정리된 데이터가 /content/정리된_서울기온.csv에 저장되었습니다.


In [ ]:
# 경로 설정
merged_file_path = '/content/서울관광객내외병합_정렬.csv'
temperature_data_path = '/content/정리된_서울기온.csv'
output_file_path = '/content/병합된_서울관광객_기온.csv'

# 파일 읽기
merged_data = pd.read_csv(merged_file_path, encoding='utf-8-sig')
temperature_data = pd.read_csv(temperature_data_path, encoding='utf-8-sig')

# '기준년월'을 기준으로 병합
merged_combined_data = pd.merge(
    merged_data, temperature_data, on='기준년월', how='left'
)

# 병합된 데이터 저장
merged_combined_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"병합된 데이터가 {output_file_path}에 저장되었습니다.")


병합된 데이터가 /content/병합된_서울관광객_기온.csv에 저장되었습니다.


In [ ]:
import pandas as pd
import os

# 디렉토리 경로 및 결과 저장 경로 설정
input_dir = '/content/drive/MyDrive/방문자수추이'  # 처리할 파일이 있는 디렉토리 경로
output_file = '/content/합쳐진_방문자수.csv'  # 결과 파일 경로

# 결과를 담을 리스트
processed_data = []

# 디렉토리 내 모든 CSV 파일 처리
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):  # CSV 파일만 처리
        file_path = os.path.join(input_dir, file_name)
        try:
            # 파일 읽기 (인코딩 변경)
            data = pd.read_csv(file_path, encoding='euc-kr')
        except UnicodeDecodeError:
            data = pd.read_csv(file_path, encoding='cp949')

        # '광역지자체' 컬럼 제거
        if '광역지자체' in data.columns:
            data = data.drop(columns=['광역지자체'])

        # '내국인'과 '외국인' 컬럼 생성
        data['내국인'] = data.apply(
            lambda row: row['방문자 수'] if '현지인' in row['방문자 구분'] or '외지인' in row['방문자 구분'] else 0,
            axis=1
        )
        data['외국인'] = data.apply(
            lambda row: row['방문자 수'] if '외국인' in row['방문자 구분'] else 0,
            axis=1
        )

        # 기준년월로 그룹화하여 합계 계산
        grouped_data = data.groupby('기준년월').agg(
            내국인=('내국인', 'sum'),
            외국인=('외국인', 'sum')
        ).reset_index()

        # 총 방문자수 추가
        grouped_data['방문자수'] = grouped_data['내국인'] + grouped_data['외국인']

        # 처리된 데이터를 리스트에 추가
        processed_data.append(grouped_data)

# 모든 파일의 데이터를 하나로 합치기
final_data = pd.concat(processed_data, ignore_index=True)

# 중복된 데이터 제거
final_data = final_data.drop_duplicates(subset='기준년월')

# 값이 없는 부분을 0으로 채우기
final_data = final_data.fillna(0)

# 결과 저장
final_data.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"모든 파일이 처리되고 결과가 {output_file}에 저장되었습니다.")


모든 파일이 처리되고 결과가 /content/합쳐진_방문자수.csv에 저장되었습니다.


In [ ]:
import pandas as pd

# 파일 경로 설정
file1 = '/content/합쳐진_방문자수.csv'  # 첫 번째 파일 경로
file2 = '/content/병합된_서울관광객_기온.csv'  # 두 번째 파일 경로

# 첫 번째 파일 로드 (방문자수 데이터) - 인코딩 설정
data1 = pd.read_csv(file1, encoding='utf-8')  # utf-8이 안되면 euc-kr로 시도

# 두 번째 파일 로드 (관광객 데이터) - 인코딩 설정
data2 = pd.read_csv(file2, encoding='utf-8')  # utf-8이 안되면 euc-kr로 시도

# 기준년월을 기준으로 데이터 병합
merged_data = data2.merge(data1[['기준년월', '내국인', '외국인']], on='기준년월', how='left')

# '구분'에 따라 방문자수 계산하여 새 열 추가
merged_data['방문자수'] = merged_data.apply(
    lambda row: row['내국인'] if row['구분'] == '내국인' else row['외국인'], axis=1
)

# 불필요한 열 삭제 (내국인, 외국인)
merged_data.drop(['내국인', '외국인'], axis=1, inplace=True)

# 병합된 데이터 확인
print(merged_data.head())

# 결과 파일 저장 (한글 인코딩 유지)
output_file = '/content/최종_병합된_데이터.csv'
merged_data.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"결과가 저장되었습니다: {output_file}")


       기준년월   구분       골프장    관광기념품    관광유원시설      기타레저      기타숙박      대형쇼핑몰  \
0  20180101  내국인  120701.0  26043.0  111061.0  355665.0  566771.0  6056574.0   
1  20180102  내국인  141696.0  28959.0  123890.0  639208.0  274243.0  6597272.0   
2  20180103  내국인  117063.0  44451.0  112622.0  546619.0  322489.0  6032241.0   
3  20180104  내국인  113794.0  50071.0   94998.0  435600.0  322619.0  5899588.0   
4  20180105  내국인  111477.0  31588.0   93509.0  432891.0  381762.0  7383704.0   

      레저용품쇼핑      렌터카  ...  캠핑장/펜션       콘도       항공운송        호텔  카지노  \
0  1134272.0  19695.0  ...   370.0  14031.0   550708.0  888759.0  0.0   
1  1100400.0  60618.0  ...     0.0   3648.0  1485335.0  550999.0  0.0   
2  1073413.0  38854.0  ...     0.0  30216.0  1642292.0  471557.0  0.0   
3  1049749.0  37543.0  ...     0.0  12677.0  1701707.0  587422.0  0.0   
4  1177246.0  60086.0  ...    50.0  11214.0  2116769.0  687969.0  0.0   

        관광총소비  평균기온(℃)  최저기온(℃)  최고기온(℃)       방문자수  
0  28498192.0     -1.3    